Import necessary frameworks.

In [2]:
from sklearn.linear_model import LogisticRegression as lr
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.model_selection import KFold
import multiprocessing as mp
import random
import warnings
warnings.filterwarnings('ignore')
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tensorflow import keras
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(rc={'figure.figsize':(16,10)}, font_scale=1.3)
import utils
import HINITE
import ourlayers
import evaluation
from tensorflow import keras

2023-06-07 15:50:02.932187: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Load the Youtube dataset.

In [4]:
data = utils.load_data("Youtube")
train_idx,val_idx,test_idx = utils.split_train_val_test(data[0][0],0.7,0.15,0.15)

Set parameters for the HINITE's architecture.

In [5]:
configs,activation = utils.config_pare_HINITE(iterations=2000,lr_rate=0.001,lr_weigh_decay=0.001,flag_early_stop=True,use_batch=512,
    rep_alpha=[0.05,0.1,0.5,1.0,1.5],flag_norm_gnn=False,flag_norm_rep=False,out_dropout=0.2,GNN_dropout=0.2,rep_dropout=0.2,inp_dropout=0.0,
    rep_hidden_layer=3,rep_hidden_shape=[128,64,64],GNN_hidden_layer=3,GNN_hidden_shape=[64,64,32], divide = True,
    head_num_att=1,out_T_layer=3,out_C_layer=3,out_hidden_shape= [128,64,32],activation = tf.nn.relu,GNN_alpha = [0.0],hete_att_size = [128,128,64]
    )

Choose the model (with different hyperparameters) by checking PEHE in the validation set, and save the result in the test set.

In [ ]:
for i in range(0,10):
    temp = [0.01,0.1,0.5,1.0, 1.5]
    cur_all_input, yf,mu1,mu0,adjs= utils.data_preparation('Youtube',i,data)
    true_ite = mu1 - mu0
    val_true_ite = true_ite[val_idx]
    print("val true ite",val_true_ite)
    val_true_ate = np.mean(true_ite[val_idx])
    test_true_ite = true_ite[test_idx]
    print("test ite",val_true_ite)
    test_true_ate = np.mean(true_ite[test_idx])
    best_pehe = 1e10
    best_err_ate = 1e10
    best_model = None
 
    for j in range(len(temp)):
        #Model_new_ASGLITE.ASGLITEModel_gcn_k3212_eps0.0__concat_split_9cross_alpha0.1rebuilding
        #test_index, cur_all_input, cur_all_y, cur_all_e = utils.data_preparation("job",i,all_data)
        cur_path = "./save_Models_HINITE/Model_Youtube1.01.0HINITE.HINITEModel_divide_True_split_" + str(i) +"rep_alpha" + str(temp[j]) 
        cur_name = "model"
        cur_model = utils.load_mymodel(cur_path, cur_name,HINITE.HINITEModel,configs[0],tf.nn.relu,adjs)
        pehe_val,err_ate_val = evaluation.evalate(Model = cur_model,inputtensor=tf.cast(cur_all_input,tf.float32),test_idx=val_idx,true_ate = tf.cast(val_true_ate,tf.float32), true_y = yf,true_ite = val_true_ite,RCT_flags = None)

        if err_ate_val < best_err_ate:
            best_err_ate  = err_ate_val
            print("best_err_ate",err_ate_val)
            #save_mymodel('./Best_Model_new_spilt'+str(i),cur_model)
            #save_mymodel(best_save_path,best_save_name,cur_model)
            #np.save("./results/best_blog_re_buildGATCFRconfig_split_"+str(i),config)
            best_model = cur_model
        elif err_ate_val == best_err_ate and  pehe_val < best_pehe:
            best_pehe = pehe_val
            #save_mymodel('./Best_Model_new_spilt'+str(i),cur_model)
            #save_mymodel(best_save_path,best_save_name,cur_model)
            #np.save("./results/best_blog_re_buildGATCFRconfig_split_"+str(i),config)
            best_model = cur_model


        cur_test_results = []
        pehe_test,err_ate_test = evaluation.evalate(Model = best_model,inputtensor=tf.cast(cur_all_input,tf.float32),test_idx=test_idx,true_ate = tf.cast(test_true_ate,tf.float32), true_y = yf,true_ite = test_true_ite,RCT_flags = None)
        cur_test_results_name = './results/test_results_Youtube_repalpha' + str(HINITE.HINITEModel)[8:-2]  +  "split_" + str(i) 
        cur_test_results.append(pehe_test)
        cur_test_results.append(err_ate_test)
        print("test pehe",pehe_test)

        utils.save_results(cur_test_results,cur_test_results_name )

Load result that you saved.

In [ ]:
# alpha 0.01,0.1,0.5,1.0,1.5 Youtube
ate_all = []
pehe_all = []
for i in range(10):
    cur_test_results_name = './results/test_results_Youtube_repalpha' + str(HINITE.HINITEModel)[8:-2]  +  "split_" + str(i)  +".npy"
    cur_result = np.load(cur_test_results_name ,allow_pickle=True)
    cur_ate =  cur_result[1]
    cur_pehe =  cur_result[0]
    ate_all.append(np.float(cur_ate))
    pehe_all.append(np.float(cur_pehe))
print(ate_all)
pehe_all

Evaluate results on the Youtube dataset.

In [ ]:
# alpha 0.01,0.1,0.5,1.0,1.5] Youtube
print(np.mean(pehe_all),np.std(pehe_all))
print(np.mean(ate_all),np.std(ate_all))